In [3]:
import re

import pandas as pd

def read_sql_results(file, regex):
    lines = open(file, 'r').readlines()
    pattern = re.compile(regex)
    lines = list(filter(lambda line: pattern.match(line), lines))

    tables = list(map(lambda line: strip(line.replace('\n', '').split('|')), lines))
    
    columns = tables.pop(0)

    return pd.DataFrame(tables, columns=columns)

def strip(lines):
    return list(map(lambda line: line.strip(), lines))

lessons = read_sql_results('data/tech_quality/lessons.txt', '.+\|.+\|.+\|.+')
participants = read_sql_results('data/tech_quality/participants.txt', '.+\|.+')
quality = read_sql_results('data/tech_quality/quality.txt', '.+\|.+')
users = read_sql_results('data/tech_quality/users.txt', '.+\|.+')

participants_to_users = participants.merge(users, left_on='user_id', right_on='id')

lessons_to_users = lessons.merge(participants_to_users.filter(items=['user_id', 'event_id', 'role']))

df = lessons_to_users.merge(quality, left_on='id', right_on='lesson_id')

df = df.filter(items=['user_id', 'tech_quality', 'subject', 'role', 'scheduled_time'])

df = df[df['role'] == 'tutor']
df = df[df['subject'] == 'phys']
df = df[df['tech_quality'] != '']

df = df.filter(items=['user_id', 'tech_quality', 'scheduled_time'])

df['date'] = pd.to_datetime(df['scheduled_time'])

df['date'] = df['date'].dt.floor('d')
df['tech_quality'] = df['tech_quality'].astype(float)

df = df.filter(items=['user_id', 'tech_quality', 'date'])

df = df.groupby(['user_id','date']).mean()

df = df.groupby('date').min()

df




,tech_quality
date,
2020-01-11,4.333333
2020-01-12,4.933333
2020-01-13,5.000000
2020-01-14,4.000000
2020-01-15,5.000000
2020-01-16,4.000000
2020-01-17,4.500000
2020-01-18,4.571429
2020-01-19,4.500000
